<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/08a_BERTEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a analizar los embeddings que devuelve BERT.

-----------------------

Tarea: responder donde dice **PREGUNTA**

## Configuración del entorno

In [ ]:
!pip install -q transformers datasets bertviz watermark

In [ ]:
import numpy as np
import pandas as pd
import torch

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from bertviz import head_view, model_view
from bertviz.neuron_view import show
from scipy.spatial.distance import cosine

In [ ]:
%load_ext watermark

In [ ]:
%watermark -vp transformers,datasets,pandas,numpy

## Qué mirás, BERT?

Como vimos, BERT fue entrenado para Masked Language Modeling (_aka_ [fill-mask](https://huggingface.co/tasks/fill-mask) en HF).

Vamos a ver cómo le va en eso.


In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert = AutoModelForMaskedLM.from_pretrained(
    "bert-base-cased", output_attentions=True, output_hidden_states=True)

In [ ]:
print(bert_tokenizer.mask_token) # <mask>

In [ ]:
input_mlm = [
    "These [MASK] are making me thirsty!",
    "These pretzels are making me [MASK]!",
    "These songs are making me [MASK]!",
    "I am [MASK]! I am without speech.",
    "[MASK] more soup for you! NEXT!",
    "I'm a [MASK] industrialist and philanthropist and a bicyclist."
]

In [ ]:
def predict_mask(input_str):
    """Tomamos el camino largo en lugar de usar pipeline
    """
    inputs = bert_tokenizer(input_str, return_tensors="pt")
    mask_index = np.where(inputs['input_ids'] == bert_tokenizer.mask_token_id)
    # .eval() to set dropout and batch normalization layers to evaluation mode
    bert.eval()
    outputs = bert(**inputs)
    top_5_predictions = torch.softmax(outputs.logits[mask_index], dim=1).topk(5)
    for i in range(5):
        token = bert_tokenizer.decode(top_5_predictions.indices[0, i])
        prob = top_5_predictions.values[0, i]
        print(f" {i+1}) {token:<20} {prob:.3f}")

In [ ]:
for x in input_mlm:
    print(x)
    out = predict_mask(x)
    print("-"*70)

**PREGUNTA 1** ¿Cómo genera el modelo las probabilidades de la palabra enmascarada?

También podemos analizar los hidden states y los attention scores.

Para esto está muy bueno [BertViz](https://github.com/jessevig/bertviz) pero también lo podemos hacer a mano.

In [ ]:
print(bert)

In [ ]:
# podemos consultar todos los pesos del modelo con:
state_dict = bert.state_dict()
list(state_dict.keys())[:5]

# o con named_parameters()

In [ ]:
print("Token embeddings tensor shape:")
print(state_dict["bert.embeddings.word_embeddings.weight"].shape)
print("Position embeddings tensor shape:")
print(state_dict["bert.embeddings.position_embeddings.weight"].shape)

**PREGUNTA 2** ¿Cómo se interpretan las dimensiones de los dos tensores anteriores?

In [ ]:
input_str = '"I voted for Obama because he was most aligned with my values", Mary said.'

In [ ]:
model_inputs = bert_tokenizer(input_str, return_tensors="pt")
bert.eval()
with torch.inference_mode():
    model_output = bert(**model_inputs)

In [ ]:
print(f"# hidden states = {len(model_output.hidden_states)}")
# initial embeddings + 12 transf. blocks

**PREGUNTA 3** ¿Qué información tiene el último hidden state del modelo?

In [ ]:
print("Size of each hidden state:")
print(model_output.hidden_states[1].shape) # (bsz, tokens, dim)

In [ ]:
print("Size of each attention tensor:")
print(model_output.attentions[0].shape) # (bsz, head, query_word, key_word)

**PREGUNTA 4** ¿Cómo se interpreta el tamaño del tensor anterior?

Veamos cómo extraer los contextual word embeddings (CWE) -- sin el [feature extractor de HF](https://huggingface.co/tasks/feature-extraction).

In [ ]:
print(type(model_output.hidden_states))
print(model_output.hidden_states[0].shape)

In [ ]:
def get_cwes(model_output):
    """Contextual embeddings como la suma de last 4 layers
    """
    # stack los 13 states en un solo tensor
    embeddings = torch.stack(model_output.hidden_states, dim=0)
    #print(embeddings.shape)
    # drop dimension de batches:
    embeddings = torch.squeeze(embeddings, dim=1)
    #print(embeddings.shape)
    # sum last 4 layers
    embeddings = embeddings[-4:].sum(dim=0)
    #print(embeddings.shape)
    return embeddings

def extract_bert_cwe(input_str, target_word):
    """Extract BERT CWE of a specific token in input_str
    """
    model_inputs = bert_tokenizer(input_str, return_tensors="pt")
    target_position = model_inputs.tokens().index(target_word)
    bert.eval()
    with torch.inference_mode():
        model_output = bert(**model_inputs)
    embedding = get_cwes(model_output)[target_position]
    return embedding

In [ ]:
# analicemos la similitud de "values" en cada contexto
input_strings = [
    '"I voted for Obama because he was most aligned with my values", Mary said.',
    'Find the values of x and y in x+y=8',
    'I believe in the values of liberal democracy.',
]

In [ ]:
word_embeddings = []
for input_ in input_strings:
    emb_ = extract_bert_cwe(input_, "values")
    word_embeddings.append(emb_)

In [ ]:
len(word_embeddings)

In [ ]:
cos_ = torch.cosine_similarity(word_embeddings[0], word_embeddings[1], dim=0).item()
print(f'Cosine sim. entre "values" de')
print(f"  {input_strings[0]}")
print(f"  {input_strings[1]}")
print(f"{cos_:.4f}")

In [ ]:
cos_ = torch.cosine_similarity(word_embeddings[0], word_embeddings[2], dim=0).item()
print(f'Cosine sim. entre "values" de')
print(f"  {input_strings[0]}")
print(f"  {input_strings[2]}")
print(f"{cos_:.4f}")

**PREGUNTA 5** ¿Por qué obtenemos un valor más alto en el segundo caso que en el primero?

De acuerdo a [What Does BERT Look At? (Clark et al, 2019)](https://arxiv.org/abs/1906.04341) las correferencias tienden a estar captadas en los heads 4-5.


In [ ]:
# attention from one token (left) to another (right)
tokens = bert_tokenizer.convert_ids_to_tokens(model_inputs.input_ids[0])
head_view(model_output.attentions, tokens)

>¿Por qué `[SEP]` recibe tanta atención?
>
>Una hipótesis es que funciona como un default cuando no aplica una función de un head (por ej, si un head representa objetos directos que prestan atención a verbos, tal vez los sustantivos en este head prestan atención a [SEP]).
>
>En definitiva, para hacer análisis, a veces conviene no tener en cuenta este token.
>
>Ver https://arxiv.org/pdf/1906.04341.pdf.

In [ ]:
# BertViz show() está buenisimo pero no funciona bien para cualquier modelo
# ver https://colab.research.google.com/drive/1hXIQ77A4TYS4y3UthWF-Ci7V7vVUoxmQ?usp=sharing#scrollTo=-QnRteSLP0Hm

## Referencias

Generales:

* [HuggingFace Docs](https://huggingface.co/docs/transformers/index)
* [HuggingFace Course](https://huggingface.co/course/)
* [HuggingFace Book](https://transformersbook.com/) (Tunstall et al, 2022)

Específicas:

* HuggingFace tutorial de [Stanford CS224n](http://web.stanford.edu/class/cs224n/)
* [Entrenar tu propio tokenizer](https://huggingface.co/docs/tokenizers/quicktour)
* [Cargar tu propio dataset](https://huggingface.co/docs/datasets/loading)
* [Streaming de large datasets](https://huggingface.co/course/chapter5/4?fw=pt)
* [HF pipeline overview](https://huggingface.co/course/chapter2/2?fw=pt)
